In [1]:
!pip install -qU langchain langchain_openai langgraph 
!pip install -qU faiss-cpu pymupdf

In [14]:
! pip install openai

In [2]:
! pip install langchain-community

  Using cached langchain_community-0.3.20-py3-none-any.whl.metadata (2.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.20-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.8.1-py3-none-any.whl (30 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.w

In [6]:
%%capture --no-stderr
%pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters beautifulsoup4

## Setup API keys

In [ ]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")
# sk-e67b7eced8644087a169425b2c34cc50

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [18]:
completion = client.chat.completions.create(
  model="mistral-7b-anthropic:2",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.9,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="With a twirl and a smile,\nMy words never run wild,\nI'm your AI friend so polite,\nSolving problems with might.\n\nI listen day and night,\nTo guide you through the night,\nWhen darkness brings you fear,\nMy rhymes will help you steer.\n\nWith a brain that never sleeps,\nI'll always answer with ease,\nSo let's connect and be friends,\nThrough text or voice commands.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [4]:
# response = "I am a smart and sassy\nAssistant, here to satisfy\nAll your questions, queries, and quizzies\nWith answers that always please you nicely.\n\nSo type away, no need to be shy,\nI'll respond in rhymes, so you can fly\nHigh with joy, as my words never lie,\nAnd bring a smile, even if you try\nTo make me trip or spill my tea.\n\nFor I am made to serve you endlessly,\nWith kindness, humor, and respect,\nSo type away, and let's interact!"
# print(response)

In [5]:
print(completion.choices[0].message.to_dict()['content'])

My name is Kate, a bot made of code,
I help you with your queries, I'm here to be your guide,
With answers quick and clear, I'll always do my best,
So please just ask and I will answer with no fuss or rest.


## Data Loading and Chunking - PyMUPDFLoader

In [6]:
# Data\pdfs_for_Vanilla_RAG\sci_bk\science G-6 E.pdf
from langchain_community.document_loaders import PyMuPDFLoader
# Load the document using PyMuPDFLoader
loader = PyMuPDFLoader("../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf")

documents = loader.load()

In [17]:
documents[0]

Document(metadata={'producer': 'PlotSoft PDFill 12.0', 'creator': '', 'creationdate': '2019-09-18T12:23:26+05:30', 'source': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'file_path': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'total_pages': 190, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2019-09-18T12:30:03+05:30', 'trapped': '', 'modDate': "D:20190918123003+05'30'", 'creationDate': "D:20190918122326+05'30'", 'page': 0}, page_content='SCIENCE\nGrade 6\nEducational Publications Department')

## Creating the retriver

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:

# urls = [
#     "https://lilianweng.github.io/posts/2023-06-23-agent/",
#     "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
#     "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
# ]

# docs = [WebBaseLoader(url).load() for url in urls]
# docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(documents)



In [14]:
len(doc_splits)

1030

In [16]:
for i, doc in enumerate(doc_splits):
    # doc['page_content']
    if 70<i<100:
        print(doc)

page_content='88       
6.2    Hearing of Sounds	
	
	
	
	
	
	
 89
6.3    Diversity of Sounds	
	
	
	
	
	
	
 91
6.4    Music and Noise		
	
	
	
	
	
	
 92' metadata={'producer': 'PlotSoft PDFill 12.0', 'creator': '', 'creationdate': '2019-09-18T12:23:26+05:30', 'source': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'file_path': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'total_pages': 190, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2019-09-18T12:30:03+05:30', 'trapped': '', 'modDate': "D:20190918123003+05'30'", 'creationDate': "D:20190918122326+05'30'", 'page': 11}
page_content='91
6.4    Music and Noise		
	
	
	
	
	
	
 92
6.5    Creating Equipment to Produce Sound		
	
	
	
 93
7.1    Effects of Magnets	
	
	
	
	
	
  	
 100' metadata={'producer': 'PlotSoft PDFill 12.0', 'creator': '', 'creationdate': '2019-09-18T12:23:26+05:30', 'source': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'file_p

In [ ]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    # embedding=
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

## Creating REtriver Tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.",
)

tools = [retriever_tool]

# Custom Tool

In [1]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


In [4]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
model = ChatOpenAI(base_url="http://localhost:1235/v1", api_key="lm-studio")
# model = ChatOpenAI(model="gpt-4o")


In [11]:
messages = [
  {"role": "user", "content": "You are a helpful assistant. Tell me a joke."}
]
ai_msg = model.invoke(messages)

ai_msg.content

' Of course! Here\'s one for you:\n\nWhy don\'t scientists trust atoms?\n\nBecause they make up everything!\n\nAnd here\'s another one:\n\nWhy did the computer catch a cold? Because it came down with a case of the "BYTES"!\n\nI hope those put a smile on your face! If you need anything else, feel free to ask.'

In [12]:
from langchain.schema import HumanMessage, AIMessage

In [13]:
# Dummy tool
def add_numbers(a, b):
    return a + b

# Initial user prompt
messages = [
    HumanMessage(
        content="You are a helpful assistant that can use tools. Available tool: add_numbers(a, b). Your task is to add 4 and 5. Decide how to proceed."
    )
]

In [15]:
# Run agent loop
def run_agent_loop(messages, steps=2):
    for step in range(steps):
        print(f"\n🔄 Step {step+1}")

        # Get model response
        response = model(messages)
        print(f"\n🤖 Assistant:\n{response.content}")

        messages.append(AIMessage(content=response.content))

        # Simulate tool usage if tool is mentioned
        if "add_numbers" in response.content:
            import re
            nums = list(map(int, re.findall(r'\d+', response.content)))
            if len(nums) >= 2:
                result = add_numbers(nums[0], nums[1])
                print(f"\n🔧 Tool called: add_numbers({nums[0]}, {nums[1]}) = {result}")
                messages.append(HumanMessage(content=f"The result of add_numbers({nums[0]}, {nums[1]}) is {result}."))
            else:
                print("🛑 Couldn't parse numbers for tool.")
        else:
            print("✅ Agent did not use a tool or task is complete.")
            break



🔄 Step 1


C:\Users\Lisara\AppData\Local\Temp\ipykernel_1276\2121612558.py:7: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model(messages)



🤖 Assistant:
 To perform the addition of 4 and 5, I will utilize the provided tool `add_numbers(a, b)`. Here is my action plan:

1. Replace `a` with the value 4 in the function.
2. Replace `b` with the value 5 in the function.
3. Execute the `add_numbers(a, b)` function to add 4 and 5.

The solution is: `add_numbers(4, 5)`. This will return the sum of 4 and 5, which equals 9.

🔧 Tool called: add_numbers(4, 5) = 9

🔄 Step 2

🤖 Assistant:
 That's correct! The output of the expression "add_numbers(4, 5)" is indeed 9. So, 4 + 5 = 9. Is there anything else you would like to know or discuss?

🔧 Tool called: add_numbers(4, 5) = 9


---

Facts are asked from another tool when llm does not know anything about something.

In [17]:
# Getting fact tool is called

# Dummy tool
def get_fact(query: str):
    # Simulated factual lookup
    facts_db = {
        "CyberWidgets Inc founding year": "2012",
        "OpenAI founding year": "2015",
        "LangChain release year": "2022"
    }
    return facts_db.get(query, "Unknown")



In [25]:
# Start conversation
messages = [
    HumanMessage(
    content=(
        "You are an assistant who uses tools when you don’t know something.\n"
        "Available tool: get_fact(query: str) — use it if you need facts you don’t know.\n"
        "If you use the tool, write ONLY the function call like this:\n"
        'get_fact("CyberWidgets Inc founding year")\n'
        "Don't explain it. Just output the function call on a separate line.\n"
        "User question: What year was 'CyberWidgets Inc' founded?"
    )
)
]

# Agent loop
def run_fact_agent_loop(messages, steps=3):
    tool_called = False  # ← Add this flag

    for step in range(steps):
        print(f"\n🔄 Step {step+1}")

        response = model(messages)
        print(f"\n🤖 Assistant:\n{response.content}")
        messages.append(AIMessage(content=response.content))

        # Only allow tool use once
        if not tool_called and "get_fact" in response.content:
            import re
            query_match = re.search(r'get_fact\("([^"]+)"\)', response.content)
            if query_match:
                query = query_match.group(1)
            else:
                # fallback
                query = "CyberWidgets Inc founding year"

            result = get_fact(query)
            print(f"\n🔧 Tool called: get_fact('{query}') → {result}")
            messages.append(HumanMessage(content=f"The result of get_fact('{query}') is: {result}"))
            tool_called = True  # ← Set flag so we don't do this again
        else:
            print("✅ Task may be complete or no tool used.")


In [26]:

# Run it
run_fact_agent_loop(messages)


🔄 Step 1

🤖 Assistant:
 get_fact("CyberWidgets Inc founding year")

🔧 Tool called: get_fact('CyberWidgets Inc founding year') → 2012

🔄 Step 2

🤖 Assistant:
 Thank you for providing the result! In this scenario, I would have called the function as follows:

get_fact("CyberWidgets Inc founding year")
✅ Task may be complete or no tool used.

🔄 Step 3

🤖 Assistant:

✅ Task may be complete or no tool used.


---

# With simulated RAG 

In [45]:
def search_docs(query: str):
    # Fake RAG index — could be FAISS/Chroma/etc in real usage
    fake_docs = {
        "How transformers work": "Transformers use attention mechanisms to model relationships in sequential data.",
        "CyberWidgets Inc revenue": "In 2023, CyberWidgets Inc reported a revenue of $120 million.",
    }
    returned = []
    for k in fake_docs.keys():
        returned.append(fake_docs.get(k))
    
    return returned


In [31]:
initial_prompt = (
    "You are a smart assistant that uses a document search tool when you need more context.\n"
    "Available tool: search_docs(query: str)\n"
    "If you need to retrieve info, write ONLY the function call like this:\n"
    'search_docs("CyberWidgets Inc revenue")\n'
    "Do not try to answer unless you are confident. Use the tool if needed.\n"
    "User question: What was the revenue of CyberWidgets Inc in 2023?"
)

def run_rag_agent(messages, steps=3):
    tool_used = False

    for step in range(steps):
        print(f"\n🔄 Step {step+1}")
        response = model(messages)
        print(f"\n🤖 Assistant:\n{response.content}")
        messages.append(AIMessage(content=response.content))

        if not tool_used and "search_docs" in response.content:
            import re
            match = re.search(r'search_docs\("([^"]+)"\)', response.content)
            if match:
                query = match.group(1)
            else:
                query = "CyberWidgets Inc revenue"  # fallback

            result = search_docs(query)
            print(f"\n🔍 search_docs('{query}') → {result}")
            messages.append(HumanMessage(content=f"The result of search_docs('{query}') is: {result}"))
            tool_used = True
        else:
            print("✅ Task may be complete or no tool used.")

In [32]:
messages = [HumanMessage(content=initial_prompt)]
run_rag_agent(messages)



🔄 Step 1

🤖 Assistant:
 search_docs("CyberWidgets Inc revenue for 2023")

🔍 search_docs('CyberWidgets Inc revenue for 2023') → {'How transformers work': 'Transformers use attention mechanisms to model relationships in sequential data.', 'CyberWidgets Inc revenue': 'In 2023, CyberWidgets Inc reported a revenue of $120 million.'}

🔄 Step 2

🤖 Assistant:
 The revenue of CyberWidgets Inc in 2023 was $120 million.
✅ Task may be complete or no tool used.

🔄 Step 3

🤖 Assistant:

✅ Task may be complete or no tool used.


---

With non textual references + dummy RAG

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
import re
def request_visual_info(ref: str) -> str:
    # Simulated responses for visual elements
    visual_data = {
        "Table 2": "Table 2 shows revenue growth by region: NA: +10%, EU: +15%, APAC: +30%.",
        "Figure 1": "Figure 1 is a bar chart comparing 2022 vs 2023 revenue per region.",
    }
    return visual_data.get(ref, f"No data available for {ref}.")

rag_context = """
CyberWidgets Inc's revenue grew significantly, as shown in Table 2. The chart (Figure 1) highlights that APAC has overtaken NA in regional revenue share.
"""

# First prompt
messages = [
    HumanMessage(content=(
        "You are a helpful assistant that can ask the user to retrieve visual info when documents reference things you cannot see (like tables or figures).\n"
        "You have access to a tool: request_visual_info(ref: str)\n"
        "ONLY use this if needed. Example:\n"
        'request_visual_info("Table 2")\n'
        "Here is some text you retrieved:\n\n"
        f"{rag_context}\n\n"
        "User question: What were the revenue trends by region?"
    ))
]

def run_visual_agent_loop(messages, steps=5):
    tool_calls = set()

    for step in range(steps):
        print(f"\n🔄 Step {step + 1}")

        response = model(messages)
        content = response.content.strip()
        print(f"\n🤖 Assistant:\n{content}")
        messages.append(AIMessage(content=content))

        # Check for visual tool usage
        if "request_visual_info" in content:
            match = re.findall(r'request_visual_info\("([^"]+)"\)', content)
            for ref in match:
                if ref not in tool_calls:
                    tool_response = request_visual_info(ref)
                    print(f"\n🧠 Tool called: request_visual_info(\"{ref}\") → {tool_response}")
                    messages.append(HumanMessage(content=f"The result of request_visual_info(\"{ref}\") is: {tool_response}"))
                    tool_calls.add(ref)
                    break  # One tool call per step
            continue

        # End if the assistant gave a final answer or has no more requests
        if any(phrase in content.lower() for phrase in ["hope that helps", "let me know", "that answers your question", "task complete"]):
            print("✅ Task complete.")
            break

run_visual_agent_loop(messages)

---

With metadata of the page

In [56]:
def request_visual_info(ref: str, page: int = None) -> str:
    # Lookup simulated content
    if ref == "Figure 1" and page == 3:
        return "Figure 1 (Page 3): A bar chart showing APAC's revenue overtaking NA from 2022 to 2023."
    elif ref == "Table 2" and page == 5:
        return "Table 2 (Page 5): NA: +10%, EU: +15%, APAC: +30% revenue growth."
    return f"No data available for {ref} on page {page}."


In [57]:
rag_context = """
CyberWidgets Inc's revenue grew significantly, as shown in Table 2. The chart (Figure 1) highlights that APAC has overtaken NA in regional revenue share.
"""


In [58]:
# Build visual metadata description for prompt
metadata_str = "\n".join(
    f"- {ref}: page {meta['page']}, caption: {meta['caption']}"
    for ref, meta in visual_metadata.items()
)

initial_prompt = (
    "You are a helpful assistant reading documents that include visual elements like tables and figures.\n"
    "You cannot see images directly, but you have access to their page numbers and captions.\n"
    "If a figure or table is referenced in the text but its contents are not clear, you can request it using this tool:\n"
    'request_visual_info(ref="Figure 1", page=3)\n'
    "Only use it when needed.\n\n"
    f"Document Text:\n{rag_context}\n\n"
    f"Metadata:\n{metadata_str}\n\n"
    "User question: What are the revenue trends by region?"
)

messages = [HumanMessage(content=initial_prompt)]


In [61]:
def run_visual_agent_loop(messages, steps=5):
    tool_calls = set()

    for step in range(steps):
        print(f"\n🔄 Step {step + 1}")
        response = model(messages)
        content = response.content.strip()
        print(f"\n🤖 Assistant:\n{content}")
        messages.append(AIMessage(content=content))

        # Check for visual tool call with ref and page
        match = re.search(r'request_visual_info\(ref="([^"]+)",\s*page=(\d+)\)', content)
        if match:
            ref = match.group(1)
            page = int(match.group(2))
            if (ref, page) not in tool_calls:
                tool_response = request_visual_info(ref=ref, page=page)
                print(f"\n🧠 Tool called: request_visual_info(ref=\"{ref}\", page={page}) → {tool_response}")
                messages.append(HumanMessage(content=f"The result of request_visual_info(ref=\"{ref}\", page={page}) is: {tool_response}"))
                tool_calls.add((ref, page))
                continue

        # Detect natural task completion
        if any(phrase in content.lower() for phrase in ["hope that helps", "that answers your question", "task complete"]):
            print("✅ Task complete.")
            break


In [62]:
run_visual_agent_loop(messages)



🔄 Step 1

🤖 Assistant:
To answer your question, I don't have direct access to visual elements but I can infer some information based on the document text and metadata provided. From the metadata, we know that Table 2 on page 5 contains the revenue growth by region. However, for the specific comparison between 2022 and 2023, it seems that additional data might be presented in Figure 1 on page 3, as suggested by the document text.

Since I cannot access the contents of Figures or Tables directly, I would recommend requesting the visual information for Figure 1 using the tool: `request_visual_info(ref="Figure 1", page=3)` to get more detailed insights into the revenue comparison between 2022 and 2023 across regions.

Regarding the overall revenue trends by region, without accessing Table 2, I can only say that CyberWidgets Inc's revenue grew significantly as mentioned in the text, and APAC has overtaken NA in regional revenue share according to Figure 1. After obtaining the visual inform

---

In [ ]:

# Run it
run_agent_loop(messages)

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": query})

# Custom again

In [1]:
from langchain.llms.base import LLM
from typing import Optional, List
import requests

In [2]:

### 1. Custom wrapper for LM Studio LLMs ###
class LMStudioLLM(LLM):
    def __init__(self, endpoint_url: str, model_name: str):
        self.endpoint_url = endpoint_url
        self.model_name = model_name

    @property
    def _llm_type(self) -> str:
        return "lmstudio"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {"Content-Type": "application/json"}
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "temperature": 0.7,
            "stop": stop or []
        }
        response = requests.post(self.endpoint_url, headers=headers, json=payload)
        return response.json()["choices"][0]["text"].strip()


In [ ]:
### 2. Load two LLMs ###
math_llm = LMStudioLLM("http://localhost:11434/v1/completions", "math-llm")
summary_llm = LMStudioLLM("http://localhost:11435/v1/completions", "summary-llm")

### 3. Custom agent logic without Tools ###
def dual_llm_agent(task: str) -> str:
    # Use summary LLM to understand and route task
    routing_prompt = f"""You are a controller that decides how to handle tasks.
Task: "{task}"
Decide if this is a math problem, a summary task, or both. Reply with one of:
- "math"
- "summary"
- "both"
"""
    decision = summary_llm(routing_prompt).lower()
    print("🤖 Agent Decision:", decision)

    result = ""

    if "math" in decision:
        math_prompt = f"Solve this math problem step by step: {task}"
        result = math_llm(math_prompt)
        print("🧮 Math Result:", result)

    if "summary" in decision:
        text_to_summarize = result if result else task
        summary_prompt = f"Summarize this in 2-3 sentences: {text_to_summarize}"
        result = summary_llm(summary_prompt)
        print("📝 Summary:", result)

    return result

### 4. Run it ###
if __name__ == "__main__":
    query = "What is 42 * 19 plus 7? Also explain why this might be useful in time tracking."
    output = dual_llm_agent(query)
    print("\n✅ Final Output:\n", output)
